In [1]:
import json
from glob import glob
from collections import Counter, OrderedDict

In [2]:
# 예측 파일들을 로드합니다.
prediction_files = glob("./results_hard/*.json")

In [3]:
print(prediction_files)

['./results_hard\\72.0800_rl_korquad_e1_npassage_e3_es40to5.json', './results_hard\\effie.json', './results_hard\\elise.json']


In [37]:
from datasets import load_from_disk
import os
import pandas as pd
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Arrow 파일 경로 설정
test_file_path = os.path.join(parent_dir, 'data', 'test_dataset')
test_dataset=load_from_disk(test_file_path)
test_dataset=test_dataset['validation']
test_df=pd.DataFrame(test_dataset)

In [12]:
# 파일을 읽어와서 predictions 리스트에 저장
def load_predictions(file_paths):
    predictions = []
    for file_path in file_paths:
        with open(file_path, "r", encoding='utf-8') as f:
            predictions.append(json.load(f))
    return predictions

# 각 문항에 대해 3개가 겹치는지, 2개가 겹치는지, 모두 다른지 확인
def check_overlap(predictions):
    all_matching = {}  # 3개가 모두 일치하는 정답
    two_matching = {}  # 2개가 일치하는 정답
    none_matching = {} # 하나도 겹치지 않는 정답

    # 모든 질문에 대해 답변 확인
    for question_id in predictions[0].keys():
        answers = [pred[question_id] for pred in predictions if question_id in pred]
        answer_counts = Counter(answers)

        # 3개의 답변이 모두 동일한 경우
        if len(answer_counts) == 1:
            all_matching[question_id] = answers[0]
        # 2개의 답변이 동일하고, 하나는 다른 경우
        elif len(answer_counts) == 2:
            most_common_answer = answer_counts.most_common(1)[0][0]
            least_common_answer = answer_counts.most_common()[-1][0]
            two_matching[question_id] = [most_common_answer, least_common_answer]
        # 모든 답변이 다른 경우
        else:
            none_matching[question_id] = answers
    
    return all_matching, two_matching, none_matching

# MRC 결과 파일 경로 설정
file_paths = glob("./results_hard/*.json")

# 파일 로드
predictions = load_predictions(file_paths)

# 각 문항에 대해 3개, 2개, 모두 다른 답변을 확인
all_matching, two_matching, none_matching = check_overlap(predictions)

# 질문 매칭을 위해 ID를 key로 한 딕셔너리 생성
id_to_question = dict(zip(test_df['id'], test_df['question']))

# 결과 출력 (질문과 함께 출력)
print("### 3개 모두 겹치는 답변 ###")
for question_id, answer in all_matching.items():
    question = id_to_question.get(question_id, "질문 없음")
    print(f"{question_id}: {answer} ({question})")

print("\n### 2개 겹치는 답변 ###")
for question_id, answers in two_matching.items():
    question = id_to_question.get(question_id, "질문 없음")
    print(f"{question_id}: {answers[0]}, {answers[1]} -----({question})")

print("\n### 모두 다른 답변 ###")
for question_id, answers in none_matching.items():
    question = id_to_question.get(question_id, "질문 없음")
    print(f"{question_id}: {answers} -----({question})")


### 3개 모두 겹치는 답변 ###
mrc-1-001113: 냉전 (용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?)
mrc-0-002191: 대통령인 빌헬름 미클라스 (돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?)
mrc-0-003951: 뉴질랜드 (마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?)
mrc-1-001272: 프랑스 (디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?)
mrc-1-000993: 유두 (단공류가 일반 포유류와 다르다는 것을 알 수 있는 신체 부위는?)
mrc-0-001283: 순조 11년(1811) (흉년이 발생하고 곳곳에 난이 일어났던 시기는?)
mrc-0-004543: 고전도성 철 (급전궤도는 보통 무엇으로 만들어져 있는가?)
mrc-0-002895: 칼라치 전방 약250km 지점 (제6군이 18일 동안이나 대기해야 했던 장소는?)
mrc-0-000535: 롭 포드 (누가 시장으로 선출되면서 트랜짓 시티 계획안이 불발되었나?)
mrc-1-001724: 노스햄프턴 교회 (1차 대각성 운동이 일어나는데 큰 영향을 끼친 설교는 어디서 진행되었는가?)
mrc-0-001606: 석조 궁륭 (슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?)
mrc-0-000266: 국민 (통대를 뽑을 수 있었던 주체는 누구인가?)
mrc-0-000032: 국방부 보고서 (미군이 충원되지 않으면 서울이 전쟁 시작 후 2주만에 함락될 것이라 밝힌 문서는?)
mrc-0-005215: 입석대(立石臺) (임경업 장군이 도를 닦은 곳으로 알려진 곳은?)
mrc-0-005407: 숙의 정씨 (화재에서 인종을 구한 공으로 귀인으로 직위가 오른 사람은?)
mrc-0-003683: 국가인권위원회 (초등학생 일기검사의 인권침해 여부를 판단하는 주체는?)
mrc-0-002835: 망자 (불보살이 천도할 대상은?)
mrc-1-001829: 화흡 (조조에게 신고한 사람도 조사해야한다고 한 것은 누구

In [19]:

from openai import OpenAI

def gpt_select_best_answer(question, answers):
    
    client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)
    prompt = f"질문: {question}\n\n가능한 답변들: {', '.join(answers)}\n\n위 답변 중에서 가장 적합한 답변을 골라서 출력해. 반드시 저 안에 있어야 하고, 답변만 출력해."
    
    # GPT-3.5 API 호출
    completion = client.chat.completions.create(
        model="chatgpt-4o-latest",  # 사용할 모델
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.5,
    )
    
    return completion.choices[0].message.content.strip()

In [21]:

# 2개 겹치는 답변에 대해 GPT로 최종 선택
two_matching_final = {}
print("\n### 2개 겹치는 답변 (GPT 선택) ###")
for question_id, answers in two_matching.items():
    question = id_to_question.get(question_id, "질문 없음")
    final_answer = gpt_select_best_answer(question, answers)
    two_matching_final[question_id] = final_answer
    print(f"{question_id}: {answers} 중 {final_answer} ({question})")

# 모두 다른 답변에 대해 GPT로 최종 선택
none_matching_final = {}
print("\n### 모두 다른 답변 (GPT 선택) ###")
for question_id, answers in none_matching.items():
    question = id_to_question.get(question_id, "질문 없음")
    final_answer = gpt_select_best_answer(question, answers)
    none_matching_final[question_id] = final_answer
    print(f"{question_id}: {answers} 중 {final_answer} ({question})")

# 최종 결과 통합
final_results = all_matching.copy()  # 3개 모두 겹치는 결과 복사
final_results.update(two_matching_final)  # 2개 겹치는 GPT 선택 결과 업데이트
final_results.update(none_matching_final)  # 모두 다른 GPT 선택 결과 업데이트

# 최종 결과를 json 파일로 저장
with open("final_mrc_results(GPT).json", "w", encoding='utf-8') as f:
    json.dump(final_results, f, indent=4, ensure_ascii=False)

print("\n최종 결과가 'final_mrc_results(GPT).json'에 저장되었습니다.")


### 2개 겹치는 답변 (GPT 선택) ###
mrc-0-005021: ['91년 체제', '대의제도'] 중 대의제도 (상류 부르주아의 정치 독점을 불러일으킨 체계는?)
mrc-1-000163: ['구미호', '선종 조사 초상에 연고를 두고 있기 때문'] 중 선종 조사 초상에 연고를 두고 있기 때문 (아시노 호가 만들어진 원인은 무엇인가?)
mrc-0-000439: ['리처드 말킨', '점쟁이'] 중 리처드 말킨 (클레어에게 오세아닉 815편을 타라고 말한 사람의 이름은?)
mrc-0-000901: ['아라라기 코요미', '오시노 메메'] 중 아라라기 코요미 (괴수로부터 메구밍을 구해준 사람은 누구인가?)
mrc-0-001326: ['‘자유’지에', '《뉴스피플》'] 중 ‘자유’지에 (이유립씨가 1970년대 중반에 본인의 글을 기고하기 시작한 곳은?)
mrc-0-003644: ['이탈리아', '백제'] 중 이탈리아 (콘스탄티누스가 군사 4만 명을 이끌고 침범한 나라는?)
mrc-0-000049: ['게임플레이', '서사'] 중 게임플레이 (루돌로지스트들이 게임에서 가장 주된 것이라 주장하는 것은?)
mrc-0-001206: ['2007년', '1849년'] 중 1849년 (펌프가 발매된 해는 언제인가?)
mrc-0-003133: ['책', '불법 어업'] 중 불법 어업 (아델리 펭귄의 목숨에 지장을 주는 사람의 업종은 무엇인가요?)
mrc-0-001058: ['룸비니 동산', '카필라 성'] 중 룸비니 동산 (석가모니 부처가 태어난 곳은 어디인가?)
mrc-0-002361: ['독립', '영토'] 중 독립 (동맹을 맺음으로써 헝가리는 자유를 비롯해 무엇을 잃었나?)
mrc-0-001086: ['라이프치히', '초계면'] 중 라이프치히 (8천 명의 시민들이 모여 대규모 시위를 벌인 지역은?)
mrc-0-000624: ['영국', '노르웨이'] 중 노르웨이 (유로니우스를 살해한 용의자가 소속된 밴드의 국적은?)
mrc-1-001280: ['미국 대통령 우드로 

     title                                            context  \
0    미국 상원  미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...   
1   인사조직관리  '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...   
2      강희제  강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...   
3   금동삼존불감  불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...   
4  계사명 사리구  동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...   

                                  question  \
0         대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?   
1                   현대적 인사조직관리의 시발점이 된 책은?   
2           강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?   
3  11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?   
4               명문이 적힌 유물을 구성하는 그릇의 총 개수는?   

                                         answers  document_id  
0        {'answer_start': [235], 'text': ['하원']}        18293  
1  {'answer_start': [212], 'text': ['《경영의 실제》']}        51638  
2        {'answer_start': [510], 'text': ['백성']}         5028  
3        {'answer_start': [625], 'text': ['중국']}        34146  
4         {'

In [40]:
from transformers import RobertaForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch

# GPU 사용이 가능한지 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# RoBERTa 모델 및 토크나이저 불러오기
model_name = "roberta-base"  # 사용할 사전 학습 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 이진 분류니까 num_labels=2

# 모델을 CUDA로 이동 (GPU 사용)
model.to(device)


C:\Users\spdlq\.conda\envs\boost-camp1\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at C:\Users\spdlq/.cache\huggingface\hub\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd80ffbac7b\config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [56]:
# 학습을 위한 `all_matching` 데이터를 Dataset 형태로 변환
def preprocess_function(examples):
    return tokenizer(examples['question'], examples['answer'], truncation=True, padding=True)

# `all_matching` 데이터를 Hugging Face Dataset으로 변환 (예: all_matching은 딕셔너리 형태로 가정)
all_matching_data = [{"question": q, "answer": a, "label": 1} for q, a in all_matching.items()]  # label=1로 설정
train_dataset = Dataset.from_list(all_matching_data)

# 평가를 위한 `eval_dataset` 설정
eval_dataset = train_dataset.train_test_split(test_size=0.1)["test"]  # 데이터의 10%를 평가용으로 사용

# 토큰화 및 데이터 전처리
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",     # 매 epoch마다 평가 진행
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

# 트레이너 설정 (eval_dataset 추가)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset  # 평가 데이터셋 추가
)

# 모델 학습
trainer.train()

# 모델을 평가 모드로 전환 (추론을 위해)
model.eval()

Map: 100%|██████████| 593/593 [00:00<00:00, 15105.93 examples/s]
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: question, answer. If question, answer are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\spdlq\.conda\envs\boost-camp1\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training ****

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [32]:
def choose_best_answer(model, tokenizer, question, answer1, answer2, device):
    # 입력 문장 토큰화 및 텐서 변환
    inputs_1 = tokenizer(question, answer1, return_tensors="pt", truncation=True, padding=True).to(device)
    inputs_2 = tokenizer(question, answer2, return_tensors="pt", truncation=True, padding=True).to(device)

    # 각 입력에 대해 BERT 모델의 출력(logits)을 얻음
    with torch.no_grad():
        outputs_1 = model(**inputs_1).logits
        outputs_2 = model(**inputs_2).logits

    # logits에서 더 큰 값을 가진 답변을 선택 (0 또는 1 예측)
    if outputs_1.argmax() == outputs_2.argmax():
        return answer1 if outputs_1[0][1] > outputs_2[0][1] else answer2
    else:
        return answer1 if outputs_1.argmax() == 1 else answer2


In [34]:
# 2개의 답변이 있는 경우 선택하기
two_matching_final = {}
for question_id, answers in two_matching.items():
    question = id_to_question.get(question_id, "질문 없음")
    answer1, answer2 = answers
    best_answer = choose_best_answer(model, tokenizer, question, answer1, answer2, device)
    two_matching_final[question_id] = best_answer
    print(f"{question_id}: {best_answer} (질문: {question})")

# 여러 개의 답변이 있는 경우 선택하기
none_matching_final = {}
for question_id, answers in none_matching.items():
    question = id_to_question.get(question_id, "질문 없음")
    # 두 개씩 비교하며 최종 답을 찾아야 함 (예: 답변 3개일 때 차례대로 비교)
    best_answer = answers[0]
    for answer in answers[1:]:
        best_answer = choose_best_answer(model, tokenizer, question, best_answer, answer, device)
    none_matching_final[question_id] = best_answer
    print(f"{question_id}: {best_answer} (질문: {question})")


mrc-0-005021: 91년 체제 (질문: 상류 부르주아의 정치 독점을 불러일으킨 체계는?)
mrc-1-000163: 구미호 (질문: 아시노 호가 만들어진 원인은 무엇인가?)
mrc-0-000439: 점쟁이 (질문: 클레어에게 오세아닉 815편을 타라고 말한 사람의 이름은?)
mrc-0-000901: 오시노 메메 (질문: 괴수로부터 메구밍을 구해준 사람은 누구인가?)
mrc-0-001326: 《뉴스피플》 (질문: 이유립씨가 1970년대 중반에 본인의 글을 기고하기 시작한 곳은?)
mrc-0-003644: 백제 (질문: 콘스탄티누스가 군사 4만 명을 이끌고 침범한 나라는?)
mrc-0-000049: 서사 (질문: 루돌로지스트들이 게임에서 가장 주된 것이라 주장하는 것은?)
mrc-0-001206: 1849년 (질문: 펌프가 발매된 해는 언제인가?)
mrc-0-003133: 책 (질문: 아델리 펭귄의 목숨에 지장을 주는 사람의 업종은 무엇인가요?)
mrc-0-001058: 카필라 성 (질문: 석가모니 부처가 태어난 곳은 어디인가?)
mrc-0-002361: 영토 (질문: 동맹을 맺음으로써 헝가리는 자유를 비롯해 무엇을 잃었나?)
mrc-0-001086: 초계면 (질문: 8천 명의 시민들이 모여 대규모 시위를 벌인 지역은?)
mrc-0-000624: 영국 (질문: 유로니우스를 살해한 용의자가 소속된 밴드의 국적은?)
mrc-1-001280: 미국 대통령 우드로 윌슨 (질문: 자신이 고안한 14개조 평화원칙을 국제 사회에 발표하고 싶어한 인물은?)
mrc-0-000504: 삼국시대 (질문: 팔과 다리가 밋밋한 형태인 신체표현은 어떤 시기를 반영한 것인가요?)
mrc-0-003763: 광배(光背)의 뒷면 (질문: 불상에 대한 정보가 적혀있는 것은?)
mrc-0-003896: 마튀랭 박사 (질문: 병으로 인해 피렌체의 수많은 사람들이 죽어갔음을 언급한 사람은?)
mrc-0-003298: 김호 (질문: 1994년 FIFA 월드컵 당시 대한민국 축구 대표팀의 감독은 누구

In [35]:
# 최종 결과 통합
final_results = all_matching.copy()
final_results.update(two_matching_final)
final_results.update(none_matching_final)

# 결과를 json 파일로 저장
with open("final_mrc_results(roberta).json", "w", encoding='utf-8') as f:
    json.dump(final_results, f, indent=4, ensure_ascii=False)

print("최종 결과가 'final_mrc_results(roberta).json'에 저장되었습니다.")


최종 결과가 'final_mrc_results(roberta).json'에 저장되었습니다.
